### Content Based Recommendation Systems
* CB are the systems that create a recommendation system by capturing similarity relationship between items according to comments description or attributes of items which users interact.
* It is used to models such as TF_IDF and Word2Vec in order to capture similarity.
* It is very powerful that a item adding newly is recommend.
* CB Filtering Approachs always provides apparent recommendation in order to use content descriprions. For example, if users have never used items having specific key words, it has no chance of recommendation.
* The reason for this, designed model is always the special the user at hand. So experience of similar users is not benefited. This tends to reduce the variety of non-ideal recommended items.
* Other disavantage in CB, although it is so powerful for present item recommendation, it is not successful in new user recommendation. In general they create hybrid model by combining content based filtering and collaborative filtering.

### What is TF-IDF?
We use to capture similarity between content (in text data such as atribute, type etc.) TF-IDF or Word2Vec model. We need to understand this topic in order to set up right recommendation system. So what is tf-idf?

TF-IDF is a weight factor which a word display important into document and had been calculated with statistics method. TF-IDF method use to a lot domains (sentiment analysis, RS, stop words filterin etc.) This method is divided into two. Fistly we will analyze Term Frequency(TF).

**Term Frequency**
It express odds a word that is found document. For example "Jandino: Whatever it Takes" is a document. Odds of "Jandino" is 1/4 (number of Jandino / number of words in document)

**Inverse Document Frequency**
It express logaritmic value of the ratio number of document and number of the word that contains document.

TF-IDF method used as multipy TF value and IDF value. (TF * IDF)

In [1]:
import numpy as np
import pandas as pd

import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
data = pd.read_csv("../../data/netflix_titles.csv")
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


I wanted to make a suggestion depends on the cast members, description and "listed_in" which contains the shows type but there is another column named "type" it is a bit confusing I know.

Drop nan values on these columns to make a proper matrix which contains linear_kernel values of selected strings.

In [3]:
data.dropna(subset=['cast', 'title', 'description', 'listed_in'], inplace=True, axis=0)
data = data.reset_index(drop=True)

Let get to know feature that we will use:

* type: media type (movie, tv show etc.)
* title: media name
* listed_in: place where it listed.
* description: information about media

In order to apply content based recommendation system, fistly we will clean text data and combine them.

In [5]:
data['listed_in'] = [re.sub(r'[^\w\s]', '', t) for t in data['listed_in']]
data['cast'] = [re.sub(',',' ',re.sub(' ','',t)) for t in data['cast']]
data['description'] = [re.sub(r'[^\w\s]', '', t) for t in data['description']]
data['title'] = [re.sub(r'[^\w\s]', '', t) for t in data['title']]

In [6]:
data["combined"] = data['listed_in'] + '  ' + data['cast'] + ' ' + data['title'] + ' ' + data['description']
data.drop(['listed_in','cast','description'], axis=1, inplace=True)
data.head()

,show_id,type,title,director,country,date_added,release_year,rating,duration,combined
0,81145628,Movie,Norm of the North King Sized Adventure,"Richard Finn, Tim Maltby","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,Children Family Movies Comedies AlanMarriott...
1,80117401,Movie,Jandino Whatever it Takes,NaN,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,StandUp Comedy JandinoAsporaat Jandino Whatev...
2,70234439,TV Show,Transformers Prime,NaN,United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids TV PeterCullen SumaleeMontano FrankWelke...
3,80058654,TV Show,Transformers Robots in Disguise,NaN,United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids TV WillFriedle DarrenCriss ConstanceZimm...
4,80125979,Movie,realityhigh,Fernando Lebrija,United States,"September 8, 2017",2017,TV-14,99 min,Comedies NestaCooper KateWalsh JohnMichaelHig...


In [7]:
# Content Similarity
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(data["combined"])

cosine_similarities = linear_kernel(matrix, matrix)

movie_title = data['title']

indices = pd.Series(data.index, index=data['title'])

In [8]:
def content_recommender(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return movie_title.iloc[movie_indices]

In [9]:
content_recommender('The Crown')

369                           Witches A Century of Murder
1829                                           London Spy
5068                                                Reign
2612                                       My Hotter Half
692       The Blue Planet A Natural History of the Oceans
3915                          The Real Football Factories
1753                                           Collateral
5474                                             Lovesick
4830                 Planet Earth The Complete Collection
2724                                         Age Gap Love
3979                                 Single Ladies Senior
4829                         Natures Great Events Diaries
3668                                       The Bomb Squad
5496                          The Frankenstein Chronicles
5060                                        Ripper Street
4776                                               Africa
1185    Oliver Stones Untold History of the United States
4362          

In [10]:
title = 'The Crown'
suggestions = content_recommender(title)

suggestions_df = pd.DataFrame(data=suggestions)
# suggestions_df.to_csv('suggestions_based_on_%s.csv'%title, index=False, header=False)

### Conclusion
* Content based RS models are powerful in new item recommendation.
* In general it is used TF-IDF and Word2Vec model while content based RS designs.
* TF-IDF method is the method which explains words frequency in document.
* It can be created a recommendation model by using cosine distance with the aid of TF-IDF weights.